In [ ]:
import numpy as np
import tensorflow as tf


def sigmoid(x):
    """
    Compute the sigmoid function.

    Parameters:
    x (float or np.ndarray): Input value(s).

    Returns:
    float or np.ndarray: Sigmoid of the input value(s).
    """
    return 1 / (1 + np.exp(-x))


def numerical_derivative(f, x):
    """
    Compute the numerical derivative of a function at a given point.

    Parameters:
    f (function): The function to differentiate.
    x (float): The point at which to evaluate the derivative.

    Returns:
    float: The numerical derivative of f at x.
    """
    dx = 1e-4
    gradf = np.zeros_like(x)

    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])

    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val + dx)
        fx1 = f(x)

        x[idx] = float(tmp_val - dx)
        fx2 = f(x)
        gradf[idx] = (fx1 - fx2) / (2 * dx)

        x[idx] = tmp_val
        it.iternext()
    return gradf


class logic_gate:

    def __init__(self, gate_name, xdata, tdata, learning_rate=0.01,
                 threshold=0.5):
        self.name = gate_name
        self.__xdata = xdata.reshape(4, 2)
        self.__tdata = tdata.reshape(4, 1)

        self.__w = np.random.rand(2, 1)
        self.__b = np.random.rand(1)

        self.__learning_rate = learning_rate
        self.__threshold = threshold

    def __loss_function(self):
        delta = 1e-7

        z = np.dot(self.__xdata, self.__w) + self.__b
        y_hat = sigmoid(z)

        return -np.sum(self.__tdata * np.log(y_hat + delta) +
                       (1 - self.__tdata) * np.log(1 - y_hat + delta))

    def err_cal(self):
        delta = 1e-7

        z = np.dot(self.__xdata, self.__w) + self.__b
        y_hat = sigmoid(z)
        return -np.sum(self.__tdata * np.log(y_hat + delta) +
                       (1 - self.__tdata) * np.log(1 - y_hat + delta))

    def train(self):
        def f(x): return self.__loss_function()
        print("initial error: ", self.err_cal())
        for step in range(20000):
            self.__w -= self.__learning_rate * \
                numerical_derivative(f, self.__w)
            self.__b -= self.__learning_rate * \
                numerical_derivative(f, self.__b)

            if step % 2000 == 0:
                print("step: ", step, "error: ", self.err_cal())

    def predict(self, input_data):
        z = np.dot(input_data, self.__w) + self.__b
        y_hat = sigmoid(z)
        if y_hat[0] > self.__threshold:
            result = 1
        else:
            result = 0
        return y_hat, result


xdata = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
tdata = np.array([[0], [0], [0], [1]])

AND = logic_gate("AND", xdata, tdata)
AND.train()
for in_data in xdata:
    (sig_val, logic_val) = AND.predict(in_data)
    print("input: ", in_data, "sigmoid: ", sig_val, "logic: ", logic_val)

xdata = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
tdata = np.array([[0], [1], [1], [1]])

OR = logic_gate("OR", xdata, tdata)
OR.train()
for in_data in xdata:
    (sig_val, logic_val) = OR.predict(in_data)
    print("input: ", in_data, "sigmoid: ", sig_val, "logic: ", logic_val)

xdata = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
tdata = np.array([[0], [1], [1], [0]])

XOR = logic_gate("XOR", xdata, tdata)
XOR.train()
for in_data in xdata:
    (sig_val, logic_val) = XOR.predict(in_data)
    print("input: ", in_data, "sigmoid: ", sig_val, "logic: ", logic_val)

fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
